# Final Project for IANNwTF 2022/23 


Learning to colorize grayscale dog pictures with the Stanford Dog Dataset.  

In [1]:
import tensorflow as tf
import numpy as np
import tensorboard
from PIL import Image
import os
from datetime import datetime
from skimage.color import rgb2lab, rgb2gray, lab2rgb
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
import random


2023-02-22 20:59:17.854682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 20:59:18.217751: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-22 20:59:18.391716: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-22 20:59:19.118439: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


2023-02-22 20:59:20.233423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 20:59:20.274195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 20:59:20.274528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 20:59:20.279874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [3]:
%load_ext tensorboard

In [4]:
# prepare data

# makes images same size and fills gaps at the edges with black pixels

def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)
    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    #image = tf.transpose(image, perm=[1, 0, 2])
    return image


In [13]:
def prepare_datasets():
    # go through folders 
    # make pairs of images + breed
    # (not needed for grayscale but might need it later)
    # divide into test and train
    base_path = "data/Images"
    lookup_table_breeds = {}
    train_img = []
    train_lbl = []
    test_img = []
    test_lbl = []
    for num,folder in enumerate(os.listdir(base_path)):
        lookup_table_breeds[folder[10:]] = num
        image_paths = os.path.join(base_path, folder)
        for count, image_path in enumerate(os.listdir(image_paths)):
            path = os.path.join(image_paths, image_path)
            if 0.9 * len(list(folder)) < count:
                # makes images same size and fills gaps at the edges with black pixels
                image = distortion_free_resize(tf.image.decode_jpeg(tf.io.read_file(path),3), (128,128))
                # convert into Lab color space
                train_img.append(rgb2lab(image))
                train_lbl.append(lookup_table_breeds[folder[10:]])

            else:
                # makes images same size and fills gaps at the edges with black pixels
                image = distortion_free_resize(tf.image.decode_jpeg(tf.io.read_file(path),3), (128,128))
                # convert into Lab color space
                test_img.append(rgb2lab(image))            
                test_lbl.append(lookup_table_breeds[folder[10:]])

    train_images = tf.data.Dataset.from_tensors(train_img)
    tf.data.Dataset.save(train_images, "saved_datasets/train_images")
    print(train_images)
    train_labels = tf.data.Dataset.from_tensors(train_lbl)
    tf.data.Dataset.save(train_labels, "saved_datasets/train_labels")
    print(train_labels)

    test_images = tf.data.Dataset.from_tensors(test_img)
    tf.data.Dataset.save(test_images, "saved_datasets/test_images")
    print(test_images)
    test_labels = tf.data.Dataset.from_tensors(test_lbl)
    tf.data.Dataset.save(test_labels, "saved_datasets/test_labels")
    print(test_labels)


In [14]:
def load_datasets():
    train_images = tf.data.Dataset.load("saved_datasets/train_images")
    train_labels = tf.data.Dataset.load("saved_datasets/train_labels")
    test_images = tf.data.Dataset.load("saved_datasets/test_images")
    test_labels = tf.data.Dataset.load("saved_datasets/test_labels")

    return train_images, train_labels, test_images, test_labels

datasets_stored = True

if datasets_stored:
    train_images, train_labels, test_images, test_labels = load_datasets()
else:
    prepare_datasets()


In [20]:
batch_size = 64

def preprocess_dataset(images, labels):
    
    # flip each image left-right with a chance of 0.3
    images = images.map(lambda x: (tf.reverse(x, axis=[-2])) if random.random() < 0.3 else (x))

    # divide into greyscale input and color output
    images = images.map(lambda x: (x[:,:,:,0], x[:,:,:,1:]))

    labels = labels.map(lambda x: tf.one_hot(x, 120))
    zipped = tf.data.Dataset.zip((images, labels))
    
    zipped = zipped.cache().shuffle(1000).batch(batch_size)

    return zipped



train_dataset = preprocess_dataset(train_images, train_labels)
test_dataset = preprocess_dataset(test_images, test_labels)


print(train_dataset)
print(test_dataset)

# the dataset has the format
# greyscale images (64,64), a and b terms from lab color space (64,64,2), onehotted labels (120)

<BatchDataset element_spec=((TensorSpec(shape=(None, 17997, 128, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 17997, 128, 128, 2), dtype=tf.float32, name=None)), TensorSpec(shape=(None, 17997, 120), dtype=tf.float32, name=None))>
<BatchDataset element_spec=((TensorSpec(shape=(None, 2583, 128, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2583, 128, 128, 2), dtype=tf.float32, name=None)), TensorSpec(shape=(None, 2583, 120), dtype=tf.float32, name=None))>


In [16]:


# or take different crops from the pictures

# show sample pictures from dataset


In [17]:
# model

# create the whole autoencoder model
# (steal from https://towardsdatascience.com/image-colorization-using-convolutional-autoencoders-fdabc1cb1dbe )




In [18]:
# training loop

# log results with tensorboard 
# save model to be able to reuse it

In [19]:
%tensorboard --logdir logs